In [1]:
import os
import numpy as np
import csv
import pandas as pd
os.chdir('/Users/renalkakhan/Documents/GitHub/CS598_DLH_Project/')
from dataset.preprocessing.word2vec_embeddings_gen import Word2VecFeatureGeneration
from dataset.preprocessing.word2vec_embeddings_gen import GloVeFeatureGeneration
from dataset.preprocessing.word2vec_embeddings_gen import FastTextFeatureGeneration
from dataset.preprocessing.word2vec_embeddings_gen import USEFeatureGeneration
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold

In [2]:
class DecisionTree:
    def __init__(self, x_train, y_train, x_test, y_test):
        self.dtc = DecisionTreeClassifier(splitter='random', random_state=42)
        self.x_train = x_train
        self.y_train = y_train
        self.x_test = x_test
        self.y_test = y_test
        # print(self.x_train.shape, self.y_train.shape, self.x_test.shape, self.y_test.shape)
        
    def train(self):
        self.dtc.fit(self.x_train, self.y_train)

    def test_and_evaluate(self):
        y_pred = self.dtc.predict(self.x_test)
        f1_macro = f1_score(self.y_test, y_pred, average='macro')
        f1_micro = f1_score(self.y_test, y_pred, average='micro')
        #print(f"Macro F1 score: {f1_macro} and Micro F1 Score {f1_micro}")
        return f1_macro, f1_micro

In [3]:
morbidities = ['Asthma', 'CAD', 'CHF', 'Depression', 'Diabetes', 'Gallstones', 'GERD', 'Gout', 'Hypercholesterolemia', 'Hypertension', 'Hypertriglyceridemia', 'OA', 'Obesity', 'OSA', 'PVD', 'Venous-Insufficiency']

column_headings = ["Morbidity Class", "DT_Macro F1", "DT_Micro F1"]

with open("./results/word-embeddings-features/performance_DT_W2V.csv", "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow([column_headings[0], column_headings[1], column_headings[2]])
    

In [4]:
all_f1_macro_scores = []
all_f1_micro_scores = []

for morbidity in morbidities:
    print(morbidity)
    train_preprocessed_df = pd.read_csv('./dataset/train/train_data_intuitive_preprocessed.csv')
    train_preprocessed_df = train_preprocessed_df[train_preprocessed_df[morbidity].isin([1.0, 0.0])]

    X, Y, words = Word2VecFeatureGeneration(train_preprocessed_df, morbidity).word2vec_matrix_gen()
    X = np.average(X, axis=1)
    # add KFold cross validation
    skf = KFold(n_splits=10, shuffle=True, random_state=42)

    f1_macro_list = []
    f1_micro_list = []
    for train_idx, val_idx in skf.split(X, Y):
        X_train_fold, Y_train_fold = X[train_idx], Y[train_idx]
        X_val_fold, Y_val_fold = X[val_idx], Y[val_idx]

        # Training RF using TF-IDF Representation
        dt_obj = DecisionTree(X_train_fold, Y_train_fold, X_val_fold, Y_val_fold)
        dt_obj.train()

        f1_macro, f1_micro = dt_obj.test_and_evaluate()

        f1_macro_list.append(f1_macro)
        f1_micro_list.append(f1_micro)

    f1_macro = np.mean(f1_macro_list)
    f1_micro = np.mean(f1_micro_list)
    print(f"Macro F1 score: {f1_macro} and Micro F1 Score {f1_micro}")

    row_heading = morbidity

    # data to be written to the CSV file
    data = [f1_macro, f1_micro]
    all_f1_macro_scores.append(f1_macro)
    all_f1_micro_scores.append(f1_micro)

    with open("./results/word-embeddings-features/performance_DT_W2V.csv", "a", newline="") as file:
        writer = csv.writer(file)
        row = [row_heading]
        row.extend(data)
        writer.writerow(row)


with open("./results/word-embeddings-features/performance_DT_W2V.csv", "a", newline="") as file:
    writer = csv.writer(file)
    row = ["Overall-Average"]
    row.extend([sum(all_f1_macro_scores)/len(all_f1_macro_scores),  sum(all_f1_micro_scores)/len(all_f1_micro_scores) ])
    writer.writerow(row)

Asthma
(66, 494, 300) (66,) Counter({0.0: 63, 1.0: 3})
Macro F1 score: 0.7187296037296038 and Micro F1 Score 0.8904761904761905
CAD
(62, 495, 300) (62,) Counter({0.0: 35, 1.0: 27})
Macro F1 score: 0.5167460317460317 and Micro F1 Score 0.5666666666666667
CHF
(12, 494, 300) (12,) Counter({1.0: 12})
Macro F1 score: 1.0 and Micro F1 Score 1.0
Depression
(66, 495, 300) (66,) Counter({0.0: 58, 1.0: 8})
Macro F1 score: 0.7594871794871795 and Micro F1 Score 0.8714285714285716
Diabetes
(63, 495, 300) (63,) Counter({1.0: 35, 0.0: 28})
Macro F1 score: 0.47861111111111104 and Micro F1 Score 0.49523809523809514
Gallstones
(66, 495, 300) (66,) Counter({0.0: 56, 1.0: 10})
Macro F1 score: 0.4232167832167832 and Micro F1 Score 0.7404761904761904
GERD
(56, 495, 300) (56,) Counter({0.0: 49, 1.0: 7})
Macro F1 score: 0.6251767676767678 and Micro F1 Score 0.8066666666666666
Gout
(67, 495, 300) (67,) Counter({0.0: 62, 1.0: 5})
Macro F1 score: 0.5861538461538461 and Micro F1 Score 0.7880952380952382
Hyperchol

In [5]:
with open("./results/word-embeddings-features/performance_DT_Glove.csv", "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow([column_headings[0], column_headings[1], column_headings[2]])
    
all_f1_macro_scores = []
all_f1_micro_scores = []

for morbidity in morbidities:
    print(morbidity)
    train_preprocessed_df = pd.read_csv('./dataset/train/train_data_intuitive_preprocessed.csv')
    train_preprocessed_df = train_preprocessed_df[train_preprocessed_df[morbidity].isin([1.0, 0.0])]

    X, Y, words = GloVeFeatureGeneration(train_preprocessed_df, morbidity).glove_matrix_gen()
    X = np.average(X, axis=1)
    # add KFold cross validation
    skf = KFold(n_splits=10, shuffle=True, random_state=42)

    f1_macro_list = []
    f1_micro_list = []
    for train_idx, val_idx in skf.split(X, Y):
        X_train_fold, Y_train_fold = X[train_idx], Y[train_idx]
        X_val_fold, Y_val_fold = X[val_idx], Y[val_idx]

        # Training RF using TF-IDF Representation
        dt_obj = DecisionTree(X_train_fold, Y_train_fold, X_val_fold, Y_val_fold)
        dt_obj.train()

        f1_macro, f1_micro = dt_obj.test_and_evaluate()

        f1_macro_list.append(f1_macro)
        f1_micro_list.append(f1_micro)

    f1_macro = np.mean(f1_macro_list)
    f1_micro = np.mean(f1_micro_list)
    print(f"Macro F1 score: {f1_macro} and Micro F1 Score {f1_micro}")

    row_heading = morbidity

    # data to be written to the CSV file
    data = [f1_macro, f1_micro]
    all_f1_macro_scores.append(f1_macro)
    all_f1_micro_scores.append(f1_micro)

    with open("./results/word-embeddings-features/performance_DT_Glove.csv", "a", newline="") as file:
        writer = csv.writer(file)
        row = [row_heading]
        row.extend(data)
        writer.writerow(row)


with open("./results/word-embeddings-features/performance_DT_Glove.csv", "a", newline="") as file:
    writer = csv.writer(file)
    row = ["Overall-Average"]
    row.extend([sum(all_f1_macro_scores)/len(all_f1_macro_scores),  sum(all_f1_micro_scores)/len(all_f1_micro_scores) ])
    writer.writerow(row)

Asthma
Macro F1 score: 0.8148407148407149 and Micro F1 Score 0.9357142857142857
CAD
Macro F1 score: 0.6790079365079364 and Micro F1 Score 0.6880952380952381
CHF
Macro F1 score: 1.0 and Micro F1 Score 1.0
Depression
Macro F1 score: 0.6457498057498057 and Micro F1 Score 0.8023809523809524
Diabetes
Macro F1 score: 0.4685064935064934 and Micro F1 Score 0.5357142857142857
Gallstones
Macro F1 score: 0.5338053613053613 and Micro F1 Score 0.7119047619047619
GERD
Macro F1 score: 0.5507070707070707 and Micro F1 Score 0.8266666666666665
Gout
Macro F1 score: 0.6072494172494172 and Micro F1 Score 0.85
Hypercholesterolemia
Macro F1 score: 0.4556926406926407 and Micro F1 Score 0.6033333333333333
Hypertension
Macro F1 score: 0.592478354978355 and Micro F1 Score 0.6666666666666667
Hypertriglyceridemia
Macro F1 score: 1.0 and Micro F1 Score 1.0
OA
Macro F1 score: 0.464125874125874 and Micro F1 Score 0.7261904761904762
Obesity
Macro F1 score: 0.47896825396825404 and Micro F1 Score 0.5666666666666667
OSA


In [6]:
with open("./results/word-embeddings-features/performance_DT_FastText.csv", "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow([column_headings[0], column_headings[1], column_headings[2]])
    
all_f1_macro_scores = []
all_f1_micro_scores = []

for morbidity in morbidities:
    print(morbidity)
    train_preprocessed_df = pd.read_csv('./dataset/train/train_data_intuitive_preprocessed.csv')
    train_preprocessed_df = train_preprocessed_df[train_preprocessed_df[morbidity].isin([1.0, 0.0])]

    X, Y, words = FastTextFeatureGeneration(train_preprocessed_df, morbidity).fasttext_matrix_gen()
    X = np.average(X, axis=1)
    # add KFold cross validation
    skf = KFold(n_splits=10, shuffle=True, random_state=42)

    f1_macro_list = []
    f1_micro_list = []
    for train_idx, val_idx in skf.split(X, Y):
        X_train_fold, Y_train_fold = X[train_idx], Y[train_idx]
        X_val_fold, Y_val_fold = X[val_idx], Y[val_idx]

        # Training RF using TF-IDF Representation
        dt_obj = DecisionTree(X_train_fold, Y_train_fold, X_val_fold, Y_val_fold)
        dt_obj.train()

        f1_macro, f1_micro = dt_obj.test_and_evaluate()

        f1_macro_list.append(f1_macro)
        f1_micro_list.append(f1_micro)

    f1_macro = np.mean(f1_macro_list)
    f1_micro = np.mean(f1_micro_list)
    print(f"Macro F1 score: {f1_macro} and Micro F1 Score {f1_micro}")

    row_heading = morbidity

    # data to be written to the CSV file
    data = [f1_macro, f1_micro]
    all_f1_macro_scores.append(f1_macro)
    all_f1_micro_scores.append(f1_micro)

    with open("./results/word-embeddings-features/performance_DT_FastText.csv", "a", newline="") as file:
        writer = csv.writer(file)
        row = [row_heading]
        row.extend(data)
        writer.writerow(row)


with open("./results/word-embeddings-features/performance_DT_FastText.csv", "a", newline="") as file:
    writer = csv.writer(file)
    row = ["Overall-Average"]
    row.extend([sum(all_f1_macro_scores)/len(all_f1_macro_scores),  sum(all_f1_micro_scores)/len(all_f1_micro_scores) ])
    writer.writerow(row)

Asthma
(66, 494, 300) (66,) Counter({0.0: 63, 1.0: 3})
Macro F1 score: 0.7732750582750583 and Micro F1 Score 0.9071428571428571
CAD
(62, 495, 300) (62,) Counter({0.0: 35, 1.0: 27})
Macro F1 score: 0.6375396825396825 and Micro F1 Score 0.6666666666666667
CHF
(12, 494, 300) (12,) Counter({1.0: 12})
Macro F1 score: 1.0 and Micro F1 Score 1.0
Depression
(66, 495, 300) (66,) Counter({0.0: 58, 1.0: 8})
Macro F1 score: 0.6276456876456875 and Micro F1 Score 0.8642857142857142
Diabetes
(63, 495, 300) (63,) Counter({1.0: 35, 0.0: 28})
Macro F1 score: 0.4549603174603174 and Micro F1 Score 0.48571428571428565
Gallstones
(66, 495, 300) (66,) Counter({0.0: 56, 1.0: 10})
Macro F1 score: 0.552004662004662 and Micro F1 Score 0.7857142857142857
GERD
(56, 495, 300) (56,) Counter({0.0: 49, 1.0: 7})
Macro F1 score: 0.5995959595959597 and Micro F1 Score 0.8300000000000001
Gout
(67, 495, 300) (67,) Counter({0.0: 62, 1.0: 5})
Macro F1 score: 0.6527622377622377 and Micro F1 Score 0.8428571428571429
Hypercholes

In [7]:
with open("./results/word-embeddings-features/performance_DT_USE.csv", "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow([column_headings[0], column_headings[1], column_headings[2]])
    
all_f1_macro_scores = []
all_f1_micro_scores = []

for morbidity in morbidities:
    print(morbidity)
    train_preprocessed_df = pd.read_csv('./dataset/train/train_data_intuitive_preprocessed.csv')
    train_preprocessed_df = train_preprocessed_df[train_preprocessed_df[morbidity].isin([1.0, 0.0])]

    X, Y, words = USEFeatureGeneration(train_preprocessed_df, morbidity).use_matrix_gen()
    X = np.average(X, axis=1)
    X = X.reshape(-1, 1)

    # add KFold cross validation
    skf = KFold(n_splits=10, shuffle=True, random_state=42)

    f1_macro_list = []
    f1_micro_list = []
    for train_idx, val_idx in skf.split(X, Y):
        X_train_fold, Y_train_fold = X[train_idx], Y[train_idx]
        X_val_fold, Y_val_fold = X[val_idx], Y[val_idx]

        # Training RF using TF-IDF Representation
        dt_obj = DecisionTree(X_train_fold, Y_train_fold, X_val_fold, Y_val_fold)
        dt_obj.train()

        f1_macro, f1_micro = dt_obj.test_and_evaluate()

        f1_macro_list.append(f1_macro)
        f1_micro_list.append(f1_micro)

    f1_macro = np.mean(f1_macro_list)
    f1_micro = np.mean(f1_micro_list)
    print(f"Macro F1 score: {f1_macro} and Micro F1 Score {f1_micro}")

    row_heading = morbidity

    # data to be written to the CSV file
    data = [f1_macro, f1_micro]
    all_f1_macro_scores.append(f1_macro)
    all_f1_micro_scores.append(f1_micro)

    with open("./results/word-embeddings-features/performance_DT_USE.csv", "a", newline="") as file:
        writer = csv.writer(file)
        row = [row_heading]
        row.extend(data)
        writer.writerow(row)


with open("./results/word-embeddings-features/performance_DT_USE.csv", "a", newline="") as file:
    writer = csv.writer(file)
    row = ["Overall-Average"]
    row.extend([sum(all_f1_macro_scores)/len(all_f1_macro_scores),  sum(all_f1_micro_scores)/len(all_f1_micro_scores) ])
    writer.writerow(row)

Asthma


2023-05-01 14:21:16.478211: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


(66, 512) (66,) Counter({0.0: 63, 1.0: 3})
Macro F1 score: 0.6155244755244754 and Micro F1 Score 0.8761904761904763
CAD
(62, 512) (62,) Counter({0.0: 35, 1.0: 27})
Macro F1 score: 0.5110714285714286 and Micro F1 Score 0.5452380952380952
CHF
(12, 512) (12,) Counter({1.0: 12})
Macro F1 score: 1.0 and Micro F1 Score 1.0
Depression
(66, 512) (66,) Counter({0.0: 58, 1.0: 8})
Macro F1 score: 0.6178554778554779 and Micro F1 Score 0.8357142857142857
Diabetes
(63, 512) (63,) Counter({1.0: 35, 0.0: 28})
Macro F1 score: 0.5238492063492063 and Micro F1 Score 0.5428571428571429
Gallstones
(66, 512) (66,) Counter({0.0: 56, 1.0: 10})
Macro F1 score: 0.7911322011322011 and Micro F1 Score 0.8976190476190476
GERD
(56, 512) (56,) Counter({0.0: 49, 1.0: 7})
Macro F1 score: 0.5528535353535353 and Micro F1 Score 0.7666666666666666
Gout
(67, 512) (67,) Counter({0.0: 62, 1.0: 5})
Macro F1 score: 0.7184615384615386 and Micro F1 Score 0.8904761904761903
Hypercholesterolemia
(59, 512) (59,) Counter({0.0: 43, 1.0